In [ ]:
!pip install keras-bert
!pip install keras-radam
!pip install wandb

---
---

In [2]:
# 라이브러리 import 및 주요 라이브러리 버전 확인 
import wandb
from wandb.keras import WandbCallback

import uuid
# uuid.uuid4() -> UUID('e4392169-4184-43ba-88c9-157d87472c82')

import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling1D, Dense, GlobalMaxPooling1D

import pandas as pd
import numpy as np  
import re
import pickle
from sklearn.model_selection import train_test_split

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

import codecs
from tqdm import tqdm
import shutil

import json

import os
import warnings
import tensorflow as tf
warnings.filterwarnings(action='ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# tf.logging.set_verbosity(tf.logging.ERROR)

from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

from keras_radam import RAdam
from keras_radam.training import RAdamOptimizer

In [3]:
tf.keras.models.save_model?

In [3]:
dktc_df = pd.read_csv("DKTC_train.csv", usecols=['class','conversation'])
immora_df = pd.read_csv("immoral_extra(oov).csv", usecols=['class','conversation'])

In [11]:
dktc_df['class'].value_counts()

기타 괴롭힘 대화      1094
갈취 대화           981
직장 내 괴롭힘 대화     979
협박 대화           896
Name: class, dtype: int64

In [12]:
immora_df['class'].value_counts()

위협대화    118140
Name: class, dtype: int64

### 섞고 정렬하기

In [4]:
for i in range(10):
    dktc_df = dktc_df.sample(frac=1).reset_index(drop=True) #많이 섞기

### class's label 숫자로 변경

In [5]:
dktc_df

,class,conversation
0,직장 내 괴롭힘 대화,야 너 일을 왜 그렇게 하냐\n어제 말씀해주신대로.\n그래 내가 그렇게 시켰다는거야...
1,갈취 대화,야 거기\n저요?\n거기 너 말고 누구 있냐?\n왜 그러시죠?\n너 돈좀 있냐?\n...
2,협박 대화,우리자식살려내 못살리면 다 죽여버릴거야\n최선을 다 하겠습니다\n하지만 지금 위급한...
3,협박 대화,너 니 자식 살리고 싶지. 그러면 빨리 널 나에게 보낸 인간 불어.\n.말못해.\n...
4,기타 괴롭힘 대화,아 찐따새끼 진짜 쟤 저렇게 맞고도 학교 잘 나온다\n그러게 쟤도 참 징하다 나 같...
...,...,...
3945,기타 괴롭힘 대화,진짜 개 웃기게 생겼네 \n뭐어?\n다운증후군?야 그거 걸린 애들은 왜 다 너같이 ...
3946,직장 내 괴롭힘 대화,수진 씨 이거 일처리를 왜 이렇게 해요?\n 네? 혹시 무슨 부분 때문인지 말씀해 ...
3947,기타 괴롭힘 대화,내일까지 레벨 몇 만들어와\n응? 나 낼 시험인데\n해오라고\n안돼.\n또 맞고싶냐...
3948,협박 대화,김사장 빌려간 돈 언제 줄거야?\n약속날짜는 이번주 주말아닙니까\n그렇긴한데 내가 ...


In [6]:
dktc_df["class"] = dktc_df["class"].astype('category')
dktc_df["class"] = dktc_df["class"].cat.codes

In [16]:
dktc_df
# 0: 갈취 대화, 1: 기타 괴롭힘 대화 , 2: 직장 내 괴롭힘 대화, 3: 협박 대화

,class,conversation
0,0,아가야\n네?\n니 손목에 팔찌 너무이쁘다\n아빠가 사주신거에요\n어머 그래?\n네...
1,0,야 임자 나 배고프다.\n나 오늘 돈 없어\n그럼 너희 집으로 가자\n왜?엄마계셔서...
2,0,현수야 내가 좀 급해서 그러는데 500만원만 주면 안돼?\n그런 큰돈이 지금 어딨어...
3,2,오 너가 새로 입사한 애냐?\n네 반갑습네다. 제가 이 회사에 새로 취직한 사람입니...
4,0,뒤로 횡령하신 전적이 있으시던데?\n어디서 뭘 봤는지 모르겠지만 잘못 봤어.\n동료...
...,...,...
3945,2,영주씨 낼 뭐해요?\n내일은 푹쉬어야죠.\n내일 나 좀 도와줘\n제가 왜 도와줘야 ...
3946,1,네가 그럼 그렇지.하긴 뭘하냐 개뿔\n나는 이렇게 될 줄 알았니\n맨날 말만 번지르...
3947,0,야 이 볼펜 나 줘\n싫은데?\n걍 줘\n내가 왜?\n아 그냥 주라고\n아 왜이래\...
3948,1,이것도 못푸냐?\n너무 어려운데\n지능이 낮어서 그럼\n말이 너무한거 아니냐?\n솔...


In [ ]:
# 0: 갈취 대화, 1: 기타 괴롭힘 대화 , 2: 직장 내 괴롭힘 대화, 3: 협박 대화
CLASS_NAMES = ["갈취 대화", "기타 괴롭힘 대화", "직장 내 괴롭힘 대화", "협박 대화"]

### train_test_split

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(dktc_df, 
                                  test_size=0.1, 
                                  random_state=5555)

In [10]:
x_train.shape, x_test.shape

((3555, 2), (395, 2))

### train_validation_split

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_val = train_test_split(x_train, 
                                  test_size=0.2, 
                                  random_state=5555)

### index 순차적으로 정렬

In [7]:
x_train.reset_index(drop="index", inplace=True)
x_val.reset_index(drop="index", inplace=True)
x_test.reset_index(drop="index", inplace=True)

In [9]:
x_train['class'].value_counts(),\
x_val['class'].value_counts(),\
x_test['class'].value_counts()

(1    811
 2    707
 0    689
 3    637
 Name: class, dtype: int64,
 1    184
 2    180
 0    177
 3    170
 Name: class, dtype: int64,
 0    115
 1     99
 2     92
 3     89
 Name: class, dtype: int64)

### korBERT 불러와서 model_extra 빌딩

In [8]:
# 문장의 최대 길이 -> 미만[초과] 0으로 채우기[자르기]
SEQ_LEN = 256

pretrained_path ="bert"
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

DATA_COLUMN = "conversation"
LABEL_COLUMN = "class"

### tokenize

In [9]:
token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        if "_" in token:
            token = token.replace("_","")
            token = "##" + token
        token_dict[token] = len(token_dict)
        #key(문자) = value(index)

In [10]:
class inherit_Tokenizer(Tokenizer):
    def _tokenize(self, text):
        if not self._cased:
            text = text
            
            text = text.lower()
        spaced = ''
        for ch in text:
            if self._is_punctuation(ch) or self._is_cjk_character(ch):
                spaced += ' ' + ch + ' '
            elif self._is_space(ch):
                spaced += ' '
            elif ord(ch) == 0 or ord(ch) == 0xfffd or self._is_control(ch):
                continue
            else:
                spaced += ch
        tokens = []
        for word in spaced.strip().split():
            tokens += self._word_piece_tokenize(word)
        return tokens

In [11]:
tokenizer = inherit_Tokenizer(token_dict)
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

### Data encoding

In [12]:
def convert_data(data_df):
    global tokenizer
    indices, targets = [], []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN) # conversation
        indices.append(ids)
        targets.append(data_df[LABEL_COLUMN][i]) # class
    items = list(zip(indices, targets))
    
    indices, targets = zip(*items)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)], np.array(targets)

def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str) # conversation


    data_x, data_y = convert_data(data_df)

    return data_x, data_y

In [13]:
train_x, train_y = load_data(x_train)
val_x, val_y = load_data(x_val)
test_x, test_y = load_data(x_test)

100%|██████████| 395/395 [00:00<00:00, 1973.48it/s]


### 모델 파인튜닝

In [14]:
wandb.login(key="fff4")

wandb: Currently logged in as: m05. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc


True

### wandb.init 에 들어갈 parameter¶

In [15]:
default_config = {
#     "n_channel_1" : 32,
#     "n_channel_2" : 64,
    "n_dense_1" : 256,
    "n_dense_2" : 768,
    "n_dense_3" : 4,  
    "learning_rate" : 5e-5,
    "epochs" : 5,
    "batch_size" : 8,
    "weight_decay" : 0.025,
#     "optimizer" : tf.keras.optimizers.Adam(learning_rate=1e-4, weight_decay=0.01),
#     "loss" : "categorical_crossentropy",
    "loss" : "sparse_categorical_crossentropy",
    "metrics" : ["accuracy"]
}

### wandb.sweep의 매개 변수¶

In [16]:
sweep_config = {
#     "name": "test_for_sweep123213", # 변동 가능. 잘 모르겠다 . wandb 페이지에서 어디에 찍히는지...
    "metric": {"name": "val_loss", "goal": "minimize"},
    "method": "random",
    "parameters": {
#         "optimizer" : {
#             'value' : 'adam'
#             },
        "batch_size" : {
            "values" : [8, 16, 32]
            },
        "learning_rate" : {
            "min" : 1e-6,
            "max" : 1e-4 # 0에 가까울 수록
            },
        "epochs" : {
            "distribution" : "int_uniform",
            "min" : 8,
            "max" : 15
            }
                    
        }
    }

### model 불러오고 설계

In [17]:
def build_model(n_dense_1, n_dense_2, n_dense_3):
    model = load_trained_model_from_checkpoint(config_path,
                                                checkpoint_path,
# 미리 학습된 모델을 사용하여 새 데이터에 대한 예측을 수행하려는 경우 "training = False
#                                                 training=True, # 새 데이터에서 모델을 계속 훈련 
                                            #     trainable=True,
                                                seq_len=SEQ_LEN)
    # trainable 설정
    for layer in model.layers[:-1]:
        layer.trainable = False

    # Unfreeze the target layer and all subsequent layers
    for layer in model.layers[-1:]:
        layer.trainable = True
        
    inputs = model.inputs[:2]
    dense = model.layers[-1].output


    x = GlobalMaxPooling1D(name="GMP")(dense)
    
    x= tf.keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(n_dense_1, activation='relu', name="1st_dense")(x)
    
    x= tf.keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(n_dense_2, activation='relu', name="2st_dense")(x)
    
    x= tf.keras.layers.BatchNormalization()(x)
    outputs = keras.layers.Dense(n_dense_3, activation='softmax', 
                               kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(x) # 


    model = keras.models.Model(inputs=inputs, outputs=outputs)
    
    return model

In [ ]:
# x = Flatten()(x)
# x = GlobalAveragePooling1D()(x)
# x = GlobalMaxPooling1D(name="Golbal_Max_Pooling")(d)

### checkpoint & earlystopping

In [18]:
path= 'multi'

In [19]:
def keras_callbacks():
    CK = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(path, "test_sweep.{epoch:02d}-{val_loss:.2f}.h5"),
                                              monitor='val_loss',
                                              save_best_only = True,
                                              save_weights_only = True)

    ES = tf.keras.callbacks.EarlyStopping(patience=1)
    
    return CK, ES

---

In [30]:
default_config

{'n_dense_1': 256,
 'n_dense_2': 768,
 'n_dense_3': 4,
 'learning_rate': 5e-05,
 'epochs': 5,
 'batch_size': 8,
 'weight_decay': 0.025,
 'loss': 'sparse_categorical_crossentropy',
 'metrics': ['accuracy']}

### model.fit 함수

In [20]:
def train():
    global default_config

    # wandb.init & config
    run = wandb.init(config = default_config, 
                     entity = "m05",
                     project = "TrainingFasle_sweep") # 변동 가능
    config = wandb.config

    # Model -> config.n_channel_1
    model = build_model(config.n_dense_1, config.n_dense_2, config.n_dense_3)    
    
    # model.compile
    model.compile(
                  optimizer = RAdamOptimizer(config.learning_rate, 
                                              config.weight_decay),
                  loss = config.loss,
                  metrics= config.metrics)
    
    #keras_callbacks
    CK, ES = keras_callbacks()
    
    # model.fit
    model.fit(train_x, train_y,
              epochs = config.epochs,
              batch_size = config.batch_size,
              validation_data = (val_x, val_y),
              callbacks = [CK, ES , 
                           WandbCallback(training_data = (test_x[:30], test_y[:30]),
                                        labels = CLASS_NAMES)]) # sweep's table에 적용시킨 하이퍼파라미터 수치를 기입이 된다.
                                        
    # model.save ㅠㅠ 저장이 안되었어.........ㅠㅠㅠㅠㅠ
#     wandb.save(os.path.join(path, "wandb.{}.h5".format(str(uuid.uuid4()))))
    
    # evaluate test
    test_loss, test_accuracy = model.evaluate(test_x, test_y, verbose=2)
    
    # wandb's 테이블쪽 column 추가하기
    wandb.log({"Test Accuracy Rate: " : round(test_accuracy * 100, 2),
               "Test Error Rate: " : round((1 - test_accuracy) * 100, 2)})
    
    run.finish() # run 종료

### model.fit

In [21]:
sweep_id = wandb.sweep(sweep_config,
                       entity = 'm05', # 고유명사
                       project = '3rd_sweep') # 변동가능

# run the sweep
wandb.agent(sweep_id,
            count=10, # 10회만 돌린다.
            function=train)

Create sweep with ID: rt2ckhhm
Sweep URL: https://wandb.ai/m05/3rd_sweep/sweeps/rt2ckhhm


wandb: Agent Starting Run: mvsjgfls with config:
wandb: 	batch_size: 32
wandb: 	epochs: 12
wandb: 	learning_rate: 9.478622318761144e-05
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/12
89/89 [==============================] - 89s 829ms/step - loss: 1.4167 - accuracy: 0.2940 - val_loss: 1.3856 - val_accuracy: 0.2293


INFO:tensorflow:Assets written to: /aiffel/aiffel/DDD/wandb/run-20230411_034758-mvsjgfls/files/model-best/assets


...

--- Logging error ---
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/wandb/agents/pyagent.py", line 288, in _run_job
    wandb.wandb_lib.config_util.save_config_file_from_dict(
  File "/opt/conda/lib/python3.9/site-packages/wandb/sdk/lib/config_util.py", line 95, in save_config_file_from_dict
    conf_file.write(data)
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/logging/__init__.py", line 1087, in emit
    self.flush()
  File "/opt/conda/lib/python3.9/logging/__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/opt/conda/lib/python3.9/threading.py", line 930, in _bootstrap
    self._bootstrap_inner()
  File "/opt/conda/lib/python3.9/thr

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
--- Logging error ---
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/wandb/agents/pyagent.py", line 288, in _run_job
    wandb.wandb_lib.config_util.save_config_file_from_dict(
  File "/opt/conda/lib/python3.9/site-packages/wandb/sdk/lib/config_util.py", line 95, in save_config_file_from_dict
    conf_file.write(data)
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/logging/__init__.py", line 1087, in emit
    self.flush()
  File "/opt/conda/lib/python3.9/logging/__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/opt/conda/lib/python3.9/threading.py", line 930, in _bootstrap
    se

--- Logging error ---
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/wandb/agents/pyagent.py", line 288, in _run_job
    wandb.wandb_lib.config_util.save_config_file_from_dict(
  File "/opt/conda/lib/python3.9/site-packages/wandb/sdk/lib/config_util.py", line 95, in save_config_file_from_dict
    conf_file.write(data)
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/logging/__init__.py", line 1087, in emit
    self.flush()
  File "/opt/conda/lib/python3.9/logging/__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/opt/conda/lib/python3.9/threading.py", line 930, in _bootstrap
    self._bootstrap_inner()
  File "/opt/conda/lib/python3.9/thr

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
--- Logging error ---
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/wandb/agents/pyagent.py", line 288, in _run_job
    wandb.wandb_lib.config_util.save_config_file_from_dict(
  File "/opt/conda/lib/python3.9/site-packages/wandb/sdk/lib/config_util.py", line 95, in save_config_file_from_dict
    conf_file.write(data)
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/logging/__init__.py", line 1087, in emit
    self.flush()
  File "/opt/conda/lib/python3.9/logging/__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/opt/conda/lib/python3.9/threading.py", line 930, in _bootstrap
    se

--- Logging error ---
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/wandb/agents/pyagent.py", line 288, in _run_job
    wandb.wandb_lib.config_util.save_config_file_from_dict(
  File "/opt/conda/lib/python3.9/site-packages/wandb/sdk/lib/config_util.py", line 95, in save_config_file_from_dict
    conf_file.write(data)
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/logging/__init__.py", line 1087, in emit
    self.flush()
  File "/opt/conda/lib/python3.9/logging/__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/opt/conda/lib/python3.9/threading.py", line 930, in _bootstrap
    self._bootstrap_inner()
  File "/opt/conda/lib/python3.9/thr

OSError: [Errno 28] No space left on device  
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ

### model's layer가 trainable인지 아닌지 확인

In [23]:
# for layer in model.layers:
#     print(layer.name, layer.trainable)

Input-Token False
Input-Segment False
Embedding-Token False
Embedding-Segment False
Embedding-Token-Segment False
Embedding-Position False
Embedding-Dropout False
Embedding-Norm False
Encoder-1-MultiHeadSelfAttention False
Encoder-1-MultiHeadSelfAttention-Dropout False
Encoder-1-MultiHeadSelfAttention-Add False
Encoder-1-MultiHeadSelfAttention-Norm False
Encoder-1-FeedForward False
Encoder-1-FeedForward-Dropout False
Encoder-1-FeedForward-Add False
Encoder-1-FeedForward-Norm False
Encoder-2-MultiHeadSelfAttention False
Encoder-2-MultiHeadSelfAttention-Dropout False
Encoder-2-MultiHeadSelfAttention-Add False
Encoder-2-MultiHeadSelfAttention-Norm False
Encoder-2-FeedForward False
Encoder-2-FeedForward-Dropout False
Encoder-2-FeedForward-Add False
Encoder-2-FeedForward-Norm False
Encoder-3-MultiHeadSelfAttention False
Encoder-3-MultiHeadSelfAttention-Dropout False
Encoder-3-MultiHeadSelfAttention-Add False
Encoder-3-MultiHeadSelfAttention-Norm False
Encoder-3-FeedForward False
Encoder-3-F

---

### Load_weights

In [25]:
model = load_trained_model_from_checkpoint(
                                            config_path,
                                            checkpoint_path,
    
# 미리 학습된 모델을 사용하여 새 데이터에 대한 예측을 수행하려는 경우 "training = False
#     training=True, # 새 데이터에서 모델을 계속 훈련 
#     trainable=True,
    
    seq_len=SEQ_LEN) # 최대 문장의 길이 256

In [23]:
def load_model(model):
     # trainable 설정
    for layer in model.layers[:-1]:
        layer.trainable = False

    # Unfreeze the target layer and all subsequent layers
    for layer in model.layers[-1:]:
        layer.trainable = True
    
    inputs = model.inputs[:3]
    dense = model.layers[-1].output
    
    x = GlobalMaxPooling1D(name="GMP")(dense)
    x = tf.keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(256, activation='relu', name="1st_dense")(x)
    
    x = tf.keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(768, activation='relu', name="2st_dense")(x)
    
    x = tf.keras.layers.BatchNormalization()(x)
    outputs = keras.layers.Dense(4, activation='softmax', 
                               kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(x) # 


    model = keras.models.Model(inputs=inputs, outputs=outputs)

    model.compile(
                optimizer = RAdamOptimizer(learning_rate=5e-4, 
                                              weight_decay=0.025),
                loss = "sparse_categorical_crossentropy", 
#                 loss =tf.keras.losses.CategoricalCrossentropy(),
                metrics= ["accuracy"])

    return model

In [26]:
multi_model = load_model(model)

In [27]:
multi_model.load_weights(os.path.join(path,"test_sweep.12-0.73.h5"))

### Load dktc_test

In [29]:
dktc_test = pd.read_csv("DKTC_test.csv", usecols=['class','conversation'])

In [111]:
dktc_test

,class,conversation
0,NaN,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
1,NaN,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
2,NaN,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
3,NaN,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,NaN,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
...,...,...
495,NaN,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...
496,NaN,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...
497,NaN,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...
498,NaN,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...


In [30]:
def predict_convert_data(data_df):
    global tokenizer
    indices= []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN) # conversation
        indices.append(ids)
        
    items = indices
    
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def predict_load_data(pd_dataframe):
    data_df = pd_dataframe    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str) # conversation

    data_x= predict_convert_data(data_df)

    return data_x

In [31]:
test_set = predict_load_data(dktc_test)

100%|██████████| 500/500 [00:00<00:00, 2142.81it/s]


In [32]:
softmax_predict = multi_model.predict(test_set, verbose=1)

16/16 [==============================] - 13s 594ms/step


In [33]:
softmax_predict

array([[0.53815144, 0.18725769, 0.1406211 , 0.13396977],
       [0.04876035, 0.10863576, 0.7442904 , 0.09831352],
       [0.06861415, 0.08023471, 0.56208897, 0.28906223],
       ...,
       [0.9460808 , 0.03791264, 0.00373516, 0.01227141],
       [0.08469672, 0.04582196, 0.12751555, 0.7419658 ],
       [0.06881882, 0.004638  , 0.01146716, 0.915076  ]], dtype=float32)

In [35]:
softmax_predict.shape

(500, 4)

In [36]:
softmax_predict[0]

array([0.53815144, 0.18725769, 0.1406211 , 0.13396977], dtype=float32)

In [39]:
softmax_predict.max(), softmax_predict.min()

(0.999323, 4.089197e-05)

---

### Softmax 값을 int 형태로 변경

In [40]:
def mean_answer_label(*preds): # * 없으면 array([1, 1, 1, 1]) 이렇게 나오넹
  preds_sum = np.zeros(preds[0].shape[0])
  
  for pred in preds:
    preds_sum += np.argmax(pred, axis=-1)
  
  return np.round(preds_sum / len(preds), 0).astype(int)

In [41]:
preds= mean_answer_label(softmax_predict)

In [42]:
preds

array([0, 2, 2, 1, 2, 3, 3, 0, 1, 0, 3, 1, 2, 2, 1, 3, 1, 1, 3, 1, 3, 1,
       1, 0, 0, 2, 1, 1, 2, 0, 0, 3, 0, 0, 3, 0, 0, 3, 3, 0, 3, 0, 2, 0,
       1, 2, 3, 1, 1, 3, 0, 1, 0, 2, 2, 0, 1, 2, 1, 2, 0, 2, 2, 2, 2, 0,
       0, 1, 0, 3, 1, 2, 1, 2, 3, 2, 2, 0, 0, 2, 2, 3, 2, 2, 1, 3, 3, 1,
       0, 2, 2, 2, 3, 1, 0, 2, 2, 2, 3, 1, 0, 0, 1, 3, 2, 2, 2, 1, 2, 3,
       3, 3, 3, 2, 0, 0, 0, 2, 0, 0, 2, 0, 3, 0, 0, 0, 2, 0, 0, 2, 1, 1,
       1, 1, 0, 3, 3, 3, 1, 1, 3, 1, 0, 0, 2, 3, 3, 3, 3, 0, 3, 1, 3, 2,
       0, 3, 3, 2, 1, 1, 3, 1, 2, 3, 2, 1, 2, 1, 1, 0, 1, 2, 2, 0, 3, 1,
       2, 2, 1, 1, 3, 1, 2, 2, 3, 1, 1, 3, 0, 3, 3, 1, 0, 1, 0, 0, 3, 1,
       3, 3, 2, 0, 3, 2, 3, 1, 2, 2, 2, 2, 3, 0, 3, 2, 0, 1, 1, 3, 3, 3,
       0, 2, 3, 3, 0, 1, 0, 3, 0, 3, 2, 3, 3, 1, 0, 0, 3, 3, 3, 1, 1, 0,
       1, 3, 2, 0, 2, 0, 2, 0, 0, 2, 3, 2, 2, 0, 0, 2, 0, 2, 1, 3, 2, 1,
       3, 1, 1, 3, 2, 0, 3, 2, 3, 2, 0, 0, 2, 1, 2, 2, 0, 3, 1, 2, 2, 2,
       0, 2, 3, 1, 0, 0, 1, 1, 2, 0, 3, 1, 0, 1, 2,

---
저스트 테스트

In [32]:
preds.max(), preds.min()

(3, 0)

In [114]:
# numpy에서 특정 숫자 counting하는 법
np.count_nonzero(preds == 1)

122

In [19]:
category_dict = {0:"기타 괴롭힘 대화", 1:"갈취 대화", 2:'직장 내 괴롭힘 대화', 3:'협박 대화'}

In [107]:
preds[5]

3

In [109]:
num_0 = []
num_1 = []
num_2 = []
num_3 = []
etc = []

for i in range(len(preds)):
    if preds[i] == 0:
        print(category_dict[preds[i]],"입니다.")
        num_0.append(i)
    elif preds[i] == 1:
        print(category_dict[preds[i]],"입니다.")
        num_1.append(i)
    elif preds[i] == 2:
        print(category_dict[preds[i]],"입니다.")
        num_2.append(i)
    elif preds[i] == 3:
        print(category_dict[preds[i]],"입니다.")
        num_3.append(i)
    else:
        print("기타 입니다.")
        etc.append(i)

기타 괴롭힘 대화 입니다.
직장 내 괴롭힘 대화 입니다.
직장 내 괴롭힘 대화 입니다.
갈취 대화 입니다.
갈취 대화 입니다.
협박 대화 입니다.
협박 대화 입니다.
기타 괴롭힘 대화 입니다.
갈취 대화 입니다.
기타 괴롭힘 대화 입니다.
협박 대화 입니다.
갈취 대화 입니다.
직장 내 괴롭힘 대화 입니다.
직장 내 괴롭힘 대화 입니다.
직장 내 괴롭힘 대화 입니다.
직장 내 괴롭힘 대화 입니다.
갈취 대화 입니다.
갈취 대화 입니다.
협박 대화 입니다.
갈취 대화 입니다.
협박 대화 입니다.
갈취 대화 입니다.
갈취 대화 입니다.
기타 괴롭힘 대화 입니다.
기타 괴롭힘 대화 입니다.
직장 내 괴롭힘 대화 입니다.
기타 괴롭힘 대화 입니다.
직장 내 괴롭힘 대화 입니다.
직장 내 괴롭힘 대화 입니다.
기타 괴롭힘 대화 입니다.
기타 괴롭힘 대화 입니다.
기타 괴롭힘 대화 입니다.
직장 내 괴롭힘 대화 입니다.
기타 괴롭힘 대화 입니다.
갈취 대화 입니다.
기타 괴롭힘 대화 입니다.
협박 대화 입니다.
협박 대화 입니다.
협박 대화 입니다.
기타 괴롭힘 대화 입니다.
협박 대화 입니다.
기타 괴롭힘 대화 입니다.
기타 괴롭힘 대화 입니다.
기타 괴롭힘 대화 입니다.
갈취 대화 입니다.
직장 내 괴롭힘 대화 입니다.
협박 대화 입니다.
갈취 대화 입니다.
갈취 대화 입니다.
협박 대화 입니다.
기타 괴롭힘 대화 입니다.
갈취 대화 입니다.
기타 괴롭힘 대화 입니다.
직장 내 괴롭힘 대화 입니다.
직장 내 괴롭힘 대화 입니다.
협박 대화 입니다.
갈취 대화 입니다.
갈취 대화 입니다.
갈취 대화 입니다.
갈취 대화 입니다.
기타 괴롭힘 대화 입니다.
협박 대화 입니다.
협박 대화 입니다.
직장 내 괴롭힘 대화 입니다.
직장 내 괴롭힘 대화 입니다.
협박 대화 입니다.
갈취 대화 입니다.
직장 내 괴롭힘 대화 입니다.
갈취 대화 입니다.
기타 괴롭힘 대화 입니다.
갈취 대화 입니다.
직장 내 괴롭힘 대화 입니다.
갈취 대화 입니다.
직장 내 괴롭힘 대화 입니다.
갈취

In [115]:
etc

[]

저스트 테스트

---

### label 달아보기

In [43]:
dktc_test['class'] = preds

In [44]:
dktc_test

,class,conversation
0,0,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
1,2,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
2,2,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
3,1,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,2,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
...,...,...
495,2,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...
496,0,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...
497,0,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...
498,3,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...


In [21]:
dktc_test.to_csv("dktc_test(sparse).csv")

### Sentence로 테스트

In [45]:
def sentence_convert_data(data):
    global tokenizer
    indices= []
    ids, segments = tokenizer.encode(data, max_len=SEQ_LEN) # conversation
    indices.append(ids)
        
    items = indices
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def category_predict(sentence):
    category_dict = {0:"기타 괴롭힘 대화", 1:"갈취 대화", 2:'직장 내 괴롭힘 대화', 3:'협박 대화'}
    global mod
    
    sentence = re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』\‘|\<\>`\'…]+", "", sentence)
    sentence = re.sub("\n+", " ", sentence)
    sentence = re.sub("\t+", " ", sentence)

    data_x = sentence_convert_data(sentence)
    predict = multi_model.predict(data_x)
    preds= mean_answer_label(predict)

    
    for i in range(len(preds)):
        if preds[i] == 0:
            print(category_dict[preds[i]],"입니다.")
        elif preds[i] == 1:
            print(category_dict[preds[i]],"입니다.")
        elif preds[i] == 2:
            print(category_dict[preds[i]],"입니다.")
        elif preds[i] == 3:
            print(category_dict[preds[i]],"입니다.")
        else:
            print("기타 입니다.")

In [46]:
category_predict('아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나보네 그럼 취소할까요 아가씨 내 여기단골이니 담에 갖다줄께 저도 알바생이라 외상안됩니다 아따 누가 떼먹는다고 그러나 갖다준다고 안됩니다 자꾸이럼 경찰불러요 아가씨 담배피는교 그건 왜 물으세요 그람 아가씨 담배 한대만 빌립시다 내 지금 지갑도 잃어버리고 기분이 그래서 그러니 여기요  아따 주는김에 한개더 주면 되겠네')

기타 괴롭힘 대화 입니다.


In [47]:
category_predict('이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 요즘 듣는 것도 들어봐 음 난 좀 별론데 좋을 줄 알았는데 아쉽네 내 취향은 아닌 듯 배고프다 밥이나 먹으러 가자 그래')

갈취 대화 입니다.


In [33]:
dktc_test['conversation'][3]

'이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 요즘 듣는 것도 들어봐 음 난 좀 별론데 좋을 줄 알았는데 아쉽네 내 취향은 아닌 듯 배고프다 밥이나 먹으러 가자 그래'

---
---
---
---

### training = False 상태의 model.summary()

In [89]:
model.summary()

Model: "model_22"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 256)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 256)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 256, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 256, 768)     1536        Input-Segment[0][0]              
___________________________________________________________________________________________

### model의 마지막 레이어에서 customize

In [ ]:
     # trainable 설정
    for layer in model.layers[:-1]:
        layer.trainable = False

    # Unfreeze the target layer and all subsequent layers
    for layer in model.layers[-1:]:
        layer.trainable = True
    
    inputs = model.inputs[:3] # model inputs이라 적힌 layer를 뽑기
    dense = model.layers[-1].output # 마지막 layer의 output만 뽑기
    
    x = GlobalMaxPooling1D(name="GMP")(dense)
    x = tf.keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(256, activation='relu', name="1st_dense")(x)
    
    x = tf.keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(768, activation='relu', name="2st_dense")(x)
    
    x = tf.keras.layers.BatchNormalization()(x)
    outputs = keras.layers.Dense(4, activation='softmax', 
                               kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(x) # 


    model = keras.models.Model(inputs=inputs, outputs=outputs)

In [17]:
multi_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 256)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 256)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 256, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 256, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________